As mentioned in [Basic syntax](/mojo/manual/basics/syntax.html), all Mojo
programs begin execution in a `main()` function, and Mojo supports two types of
functions: `def` and `fn` functions.

You can use either declaration with any function, including the `main()`
function, but they have different behaviors as described on this page.

**Note:** Functions declared inside a
[`struct`](/mojo/manual/basics/structs.html) are called "methods," but they
have all the same qualities as "functions" described here.

## `def` functions

Mojo supports a lot of Python's syntax and behavior, including the `def`
function declaration. A Mojo `def` function is designed to provide some of the
same dynamism and flexibility as Python `def` functions.

For example, this function works the same in Python and Mojo:

In [1]:
def greet(name):
    return "Hello, " + name + "!"

What you need to know about a `def` function is the following:

- Arguments don't require a type specifier. By default, all arguments are
  passed as an [`object`](/mojo/stdlib/builtin/object.html#object), which can
  represent most (eventually all) types in the Mojo standard library. For
  example, the above `greet()` function can receive a string, an integer, a
  list, and other types in the `name` argument. It's the function's
  responsibility to detect the types it does or doesn't support.

- Return types don't need to be declared either, so the function can return
  anything (or nothing).

- Arguments passed into a `def` function are **mutable** variables, using
  **value semantics** (by default). This means a `def` function receives a copy
  of the argument data, so it can modify arguments inside the function and those
  changes are visible only inside the function (unless it returns the values).
  (You can instead make arguments immutable, or mutable with reference
  semantics, but that's a topic covered in [value
  ownership](/mojo/manual/values/).)

The ability for `def` functions to use undeclared argument types and return
types provides simplicity when coding, and flexibility to handle different data
types with one function. However, this lack of type enforcement can lead to
runtime errors when a function receives or returns an unexpected type. It also
adds a runtime performance cost because Mojo must discover the data type at
runtime.

However, you can optionally declare types for some or all arguments and return
values in a `def` function, and Mojo enforces them at compile time. (See
the next section for the type declaration syntax.)

**Note:** If you know Python well, it's important to recognize that Mojo's
value semantics in a `def` function (passing arguments by value) is a
significant divergence from Python. In Python, it only _appears_ that arguments
are passed by value. The only arguments that Python actually passes by value
are immutable types, such as integers and strings. Python passes all mutable
types as object pointers (also called "pass by assignment" because it assigns
the object pointer to the argument name). If the function assigns a new value
to the argument variable, it behaves like value semantics, because the variable
now points to a different object. However, if instead of reassigning the
variable, you modify the object's value, then Python behaves like reference
semantics. For example, if your Python function receives a list and then calls
`.append()` on that list, this actually modifies the original object. In Mojo,
that won't happen because the `def` function receives its own copy of the list
(and it's a deep copy)—that is, unless you modify the [argument
convention](/mojo/manual/values/ownership.html).

## `fn` functions

As far as a caller is concerned, `def` and `fn` functions are interchangeable.
There is nothing a `def` can do that an `fn` can't (and vice versa). The
difference is that, compared to a `def` function, an `fn` function is more
strict on the inside:

- Arguments must specify their type, such as `Int` or `String` (except for the
  `self` argument in [struct methods](/mojo/manual/basics/structs.html)). If you
  want a more dynamic behavior for some arguments, you can specify the type as
  the generic `object`.

- Return values must also specify a type. If you don't specify a return type, it
  defaults to `None` (if the function doesn't return anything, then you don't
  have to specify `None`). The return type may also be a generic `object`.

- If the function raises an exception, it must be explicitly declared with the
  `raises` keyword. (A `def` function does not need to declare exceptions.)

- Argument passed into an `fn` function are **immutable** variables (the
  function can read the values but cannot change them). This catches accidental
  mutations, and allows the function to receive non-copyable types as arguments.
  (You can instead make arguments mutable, using either reference or value
  semantics, but that's a topic covered in [value
  ownership](/mojo/manual/values/ownership.html).)

By enforcing all these type checks, using the `fn` function helps avoid a
variety of potential runtime errors. It also improves performance compared to
the dynamic typing in a `def` function, because there's no overhead processing
required to figure out what data types to use at runtime—the types are fixed at
compile time.

So, a reimplementation of the `def greet()` function using `fn` looks like this:

In [6]:
from String import String

fn greet(name: String) -> String:
    return String("Hello, ") + name + String("!")

Notice that the `name` argument is now specified as a `String`, and the return
type (also `String`) is specified with an arrow `->`.

Because the `String` return type is enforced, we must concatenate the text
together by instantiating new `String` objects for each piece of text.

Another difference is—although we haven't discussed
[variables](/mojo/manual/basics/variables.html) yet—all local variables in an
`fn` function must be declared using either the `var` or `let` keyword (meaning
mutable or immutable, respectively). Whereas, in a `def` function, you don't
need either declaration and variable names are mutable by default.

**Note:** You can also use type declarations in a `def` function, the same way
as shown here with `fn` and Mojo will enforce them. (The caveat is that, when
using a `def` function, Mojo will not complain if you forget to declare any
types.)

## Overloaded functions

If a `def` function does not specify argument types, then it can accept any
data type and decide how to handle each type internally. This is nice when you
want expressive APIs that just work by accepting arbitrary inputs, but it also
has drawbacks mentioned above.

Because all `fn` functions must specify argument types, if you want a given
function to work with different data types, you need to implement separate
versions of the function that each specify different argument types. This is
called "overloading" a function.

For example, here's an overloaded `add()` function that can accept either
`Int` or `String` types:

In [3]:
from String import String

fn add(x: Int, y: Int) -> Int:
    return x + y

fn add(x: String, y: String) -> String:
    return x + y

print(add("1", "2"))

12


If you pass anything other than `Int` or `String` to the `add()` function,
you'll get a compiler error. That is, unless `Int` or `String` can implicitly
cast the type into their own type—for example, `String` includes an overloaded
version of its constructor (`__init__()`) that accepts a `StringLiteral` value
(this is actually how the above call to `add()` works at all, because wrapping
a string in quotes creates a `StringLiteral`, not a `String`).

When resolving an overloaded function call, the Mojo compiler tries each
candidate function and uses the one that works (if only one version works), or
it picks the closest match (if it can determine a close match), or it reports
that the call is ambiguous (if it can’t figure out which one to pick). In the
latter case, you can resolve the ambiguity by explicitly casting your input to
an acceptable argument type.

When resolving an overloaded function, Mojo does not consider the return type
or other contextual information at the call site—only the argument types affect
which function is selected.

Overloading also works with combinations of both `fn` and `def` functions.
For example, you could define multiple `fn` function overloads and then one
or more `def` versions that don't specify all argument types, as a fallback.

**Note:** Although we haven't discussed
[parameters](/mojo/manual/metaprogramming/parameters.html) yet (they're
different from function arguments), you can also overload functions based on
parameter types.